In [ ]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Summarize:\n\n parse url `url` to a dictionary of arguments `url_data` using urllib.parse.urlparse to get dictionary of url arguments.  parse url query string `url_data.query` into dictionary `arg_dict` and store it in dictionary `args_list`.  convert a dictionary `arg_dict` into a dictionary of values for the key `arg` and value `value` from a dictionary to a dictionary'.",
  temperature=0.7,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

In [ ]:
response

In [ ]:
def prepare_dataset_code_summary(examples, tokenizer, max_seq_length=None):
    max_seq_length = tokenizer.model_max_length if max_seq_length is None else max_seq_length

    examples_with_prompts = list(map(lambda x: "Summarize Python: " + x, map(lambda x: x, examples['code'])))
    tokenized_examples = tokenizer(
        examples_with_prompts,
        truncation=True,
        max_length=max_seq_length,
        padding='max_length',
        return_tensors = 'pt'
    )

    return tokenized_examples

In [ ]:
KeyDataset(dataset, 'code')[0]

In [ ]:
path_to_single_line_model = './single_line_summarization_model/'

pipe = pipeline(
    task = "summarization",
    model = "./single_line_summarization_model/"
)

print("Preprocessing data... (this takes a little bit, should only happen once per dataset)")
dataset = datasets.load_dataset('json', data_files="./codesearchnet-corpus/python/final/jsonl/train/python_train_0.jsonl")
dataset = dataset['train']

# print(dataset[0]['code'])

for out in tqdm(pipe(KeyDataset(dataset, 'code'))):
    print(out)

In [ ]:
import openai
import datasets
from transformers import AutoTokenizer, DataCollatorForSeq2Seq,\
    AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, HfArgumentParser
# from helpers import prepare_dataset_code_summary, compute_rouge_and_bleu
from tqdm import tqdm
import os
import json

In [ ]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

In [ ]:
# Initialize the model and tokenizer from the specified pretrained model/checkpoint
pipe = pipeline(
    task = "summarization",
    model = "./single_line_summarization_model/"
)

In [ ]:
example = ["def get_url_args(url):\n    url_data = urllib.parse.urlparse(url)\n", "    arg_dict = urllib.parse.parse_qs(url_data.query)\n", "    return arg_dict"]

In [ ]:
pipe(dataset[0]['code'])

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from transformers import Pipeline


class SummaryGenerationPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, maybe_arg=2):
        model_input = Tensor(inputs["input_ids"])
        return {"model_input": model_input}

    def _forward(self, model_inputs):
        # model_inputs == {"model_input": model_input}
        outputs = self.model(**model_inputs)
        # Maybe {"logits": Tensor(...)}
        return outputs

    def postprocess(self, model_outputs):
        best_class = model_outputs["logits"].softmax(-1)
        return best_class